# Apertura de cuentas en Kaggle


https://www.kaggle.com/


Verificar SMS para tener acceso a GPUs

Para SMS verificado sin usar el telefono propio o para abrir varias cuentas :)

https://www.freereceivesms.com/


# Generacion de texto con RNN

## Imports

In [ ]:
import os
import time
import requests
import pickle
import numpy as np
import tensorflow as tf
import re
import warnings
warnings.filterwarnings('ignore')

## Lectura de la data

Primero vayan al menu File => Upload data

Suben ahi la data, en este caso el archivo pickole que tiene los txt pero pueden subir lo que quieran

In [ ]:
import os
print(os.listdir('../input/librosmises/'))


In [ ]:
# data Mises
with open('../input/librosmises/mises_pickle.pickle', 'rb') as file:
    data = pickle.load(file)

In [ ]:
text = data
text = re.sub(r'[^\w\s,.ñ?¿]', '', text)
text = text.lower()
characters_to_replace = ['ª', 'º', 'à', 'ä', 'ç', 'è', 'ê', 'ë', 'î', 'ô', 'ö', 'ü']

for char in characters_to_replace:
    text = text.replace(char, '')

print(f"Length of text: {len(text)} characters")

In [ ]:
import pprint
pprint.pprint(text[:500])

#### Caracteres unicos

In [ ]:
vocab = sorted(set(text))
print(f"{len(vocab)} caracteres unicos")

## Procesamiento del texto

### Vectorizacion del texto

Antes de entrenar debemos convertir el texto en numeros

Usando tf.keras.layers.StringLookup, se puede convertir cada carácter en un ID numérico (y viceversa). Solo necesita que el texto se divida en tokens primero.

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary = list(vocab), mask_token=None
)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary = ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# tokenizo un texto
example_text = 'hola que tal?'
chars = tf.strings.unicode_split(example_text, input_encoding="UTF-8")

ids = ids_from_chars(chars)
print(ids)

In [ ]:
# hago en encode de los tokens 
print(text_from_ids(ids))

## Genero los IDs de todo el dataset

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode("utf-8"), end='')

## Genero los lotes

Dividimos el texto en secuencias de ejemplo.

Cada secuencia de entrada contendrá seq_length caracteres del texto.

Dividimos el texto en fragmentos de seq_length+1. (El +1 es porque como veremos luego necesitamos una secuencia y su target q es lka misma secuencia desplazada 1 caracter a la derecha)

In [ ]:
seq_length = 150
examples_per_epoch = len(text) // (seq_length + 1)
examples_per_epoch

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy().decode())

## Desfaso las secuencias

Para cada secuencia de entrada, sus targets contienen la misma longitud de texto, desplazada un carácter a la derecha.

Por ejemplo, supongamos que seq_length es 4 y nuestro texto es "El taco no..."
* La secuencia de entrada sería "El ta"
* Y la secuencia objetivo sería "l tac".

usamos tf.data.Dataset.from_tensor_slices para convertir el vector de texto en un flujo de índices de caracteres.

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
# Ejemplo para visualizarlo
[''.join(x) for x in split_input_target(list("El taco no, hace la personal"))]

In [ ]:
# Lo hago para mi dataset
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    pprint.pprint(f"Input: {text_from_ids(input_example).numpy().decode()}")
    pprint.pprint(f"Target: {text_from_ids(target_example).numpy().decode()}")

## Creo los lotes de entrenamiento

In [ ]:
dataset = sequences.map(split_input_target)

BATCH_SIZE = 192

BUFFER_SIZE = 10000
dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

## Instancio el modelo

In [ ]:
# Settings:
vocab_size = len(vocab)
embedding_dim = 192
rnn_units = 2048

Capas del modelo

* tf.keras.layers.Embedding: La capa de entrada. Una tabla de búsqueda entrenable que mapeará cada ID (q representa a un carácter) a un vector con dimensiones "embedding_dim"

* tf.keras.layers.GRU: Un tipo de RNN con tamaño units=rnn_units, se podria usar una LSTM aca

* tf.keras.layers.Dense: La capa de salida, con vocab_size salidas. Emite un logit para cada carácter en el vocabulario. Estos son la log-verosimilitud de cada carácter según el modelo. O sea la probabilidad de que sea cualquiera de los caracteres posibles del vocabulario

The class below does the following:
- Heredamos desde tf.keras.Model
- En el constructor definimos las 3 capas del modelo
- Definimos el paso hacia adelante utilizando las capas definidas en el constructor.

In [ ]:
class MyModelRNN(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x


class MyModelLSTM(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(rnn_units, return_sequences=True, return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs, training=training)
        if states is None:
            states = self.lstm.get_initial_state(x)  # LSTM tiene 2 estados.
        x, h, c = self.lstm(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, [h, c]
        else:
            return x

In [ ]:
model = MyModelLSTM(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
    )

El codigo muestra la forma de las predicciones, ejemplo: 64 tamaño del lote, 150 de longitud de cadena y 48 de tamaño de vocabulario que son las posible predicciones

In [ ]:
# Vemos el tamaño del modelo
model.summary()

## Entrenamiento del modelo

Vamos a tratar la RNN como un simple problema de clasificacion.

Dado el estado anterior de la RNN y la entrada en este paso de tiempo, predice la clase del siguiente carácter

## Funcion de perdida

Usaremos la función de pérdida estándar tf.keras.losses.sparse_categorical_crossentropy

En este caso se aplica a la última dimensión de las predicciones

Debido a que nuestro modelo devuelve logits, necesitamos establecer el indicador from_logits






In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Tamaño de las predicciones: ", example_batch_predictions.shape)
print("Perdida media: ", example_batch_mean_loss.numpy())

Como recien inicializamos el modelo, no aprendio nada aun, por lo tanto no debería estar demasiado seguro de sí mismo, los logits de salida deberían tener todas magnitudes similares. Es decir, es un mono tirando probabilidades sin tener idea y todos los caracteres deberiajn tener mas o menos la misma probabilidad o dentro de un rango acotado de probabilidades

Para confirmar esto, calculamos exponencial de la pérdida media, que deberia ser aproximadamente igual al tamaño del vocabulario.

Una pérdida mucho más alta significa que el modelo está seguro de sus respuestas incorrectas y está mal inicializado

In [ ]:
tf.exp(example_batch_mean_loss).numpy()

### Otras funciones de pérdida

* tf.keras.losses.BinaryCrossentropy: Esta sería una elección apropiada para un problema de clasificación binaria

* tf.keras.losses.MeanSquaredError: Esta se usaría típicamente para problemas de regresión

* tf.keras.losses.CategoricalCrossentropy: Esta sería una elección apropiada para un problema de clasificación multiclase en el que las etiquetas han sido codificadas en one-hot

## Optimizador

In [ ]:
model.compile(optimizer="adam", loss=loss)

### Otros Optimizadores

* tf.keras.optimizers.SGD: Este es el optimizador de descenso de gradiente estocástico. Es más simple que Adam y ha estado en uso por más tiempo.

* tf.keras.optimizers.RMSprop: Este optimizador también es una opción popular y funciona bien en muchos casos.

* tf.keras.optimizers.Adagrad y tf.keras.optimizers.Adadelta son otras dos opciones.

## Configurar checkpoints

Usamos `tf.keras.callbacks.ModelCheckpoint` para guardar los checkpoints durante el entrenamiento

In [ ]:
checkpoint_dir = "./training_checkpoints"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

## Entrenamos el modelo

In [ ]:
from tqdm.notebook import tqdm

class TqdmProgressCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epochs = self.params['epochs']

    def on_epoch_begin(self, epoch, logs=None):
        self.current_epoch = epoch  # Guarda el número de la época actual
        self.pbar = tqdm(total=self.params['steps'], unit='step')

    def on_batch_end(self, batch, logs=None):
        self.pbar.update()
        self.pbar.set_description(f"Epoch {self.current_epoch+1}/{self.epochs} \
                 Loss: {logs.get('loss'):.3f}")


    def on_epoch_end(self, epoch, logs=None):
        self.pbar.close()

callbacks = [TqdmProgressCallback()]   # checkpoint_callback
history = model.fit(dataset, epochs=6, callbacks=callbacks, verbose=0)

In [ ]:
folder_path = "/kaggle/working/training_checkpoints"
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        os.remove(file_path)

In [ ]:
with open('modelLSTM.pickle', 'wb') as file:
    pickle.dump(model, file)

## Generar texto

In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        predicted_logits = predicted_logits + self.prediction_mask
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        predicted_chars = self.chars_from_ids(predicted_ids)
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.05)

start = time.time()
states = None
next_char = tf.constant(["los paises crecen porque "])
result = [next_char]

for n in range(180):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

## Exportar el genrador de texto

In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["La inflacion "])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))